In [384]:
import pandas as pd
import json

In [385]:
path = 'noTimeCriticality/routing_output.json'
path2 = 'noTimeCriticality/sim_preprocessed_config.json'

routing_data_json = open(path)
routing_data_json = json.load(routing_data_json)

input_data_json = open(path2)
input_data_json = json.load(input_data_json)

In [386]:
# Extracting data from the JSON structure
routing_data = []
for day in routing_data_json["days"]:
    day_routes = []
    for vehicle in day["vehicles"]:
        day_routes.append(vehicle["route"])
    routing_data.append(day_routes)

# Creating a pandas DataFrame
df = pd.DataFrame(routing_data, columns=[f"Vehicle_{i+1}" for i in range(len(routing_data[0]))])
df = df.applymap(lambda x: [394 if elem == 395 else elem for elem in x])


In [387]:
df

,Vehicle_1,Vehicle_2,Vehicle_3,Vehicle_4,Vehicle_5,Vehicle_6,Vehicle_7,Vehicle_8,Vehicle_9
0,[],[],[],[],"[394, 129, 394]",[],[],[],"[394, 122, 394]"
1,[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],"[394, 103, 394]"
3,[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...
246,[],[],[],[],"[394, 182, 394]",[],[],"[394, 45, 394]",[]
247,[],[],[],[],"[394, 182, 394]",[],[],[],"[394, 311, 394]"
248,[],[],[],[],[],[],"[394, 349, 394]",[],"[394, 178, 394]"
249,[],[],[],[],[],[],[],[],[]


In [388]:
# Extract pickup sites data

type_mapping = {
    1.0 : "Grass/straw",
    2.0 : "Dry manure",
    3.0 : "Slurry manure"
}

pickup_sites_data = [
    {
        "Location_index": site["location_index"],
        "Type": type_mapping.get(site["type"], site["type"]),  # Use the mapping or original value
        "Latitude": site["lonlats"][1],
        "Longitude": site["lonlats"][0],
        "Annual input" : round(site["capacity"],2)
    }
    for site in input_data_json["pickup_sites"]
]

# Extract depots data
depots_data = [
    {
        "Location_index": 394,
        "Type": "Biogas plant",
        "Latitude": depot["lonlats"][1],
        "Longitude": depot["lonlats"][0]
    }
    for depot in input_data_json["depots"]
]

# Create DataFrames
pickup_sites_df = pd.DataFrame(pickup_sites_data)
depots_df = pd.DataFrame(depots_data)

coord_df = pd.concat([pickup_sites_df, depots_df], ignore_index = True)

In [389]:
coord_df

,Location_index,Type,Latitude,Longitude,Annual input
0,0,Dry manure,60.945884,23.352659,117.55
1,1,Dry manure,60.905353,23.329982,106.00
2,2,Dry manure,60.913446,23.435890,129.51
3,3,Dry manure,60.987171,23.410918,117.80
4,4,Dry manure,60.956071,23.491162,733.48
...,...,...,...,...,...
390,390,Grass/straw,60.595201,23.277416,414.61
391,391,Grass/straw,61.266603,23.503156,157.52
392,392,Grass/straw,60.887524,22.669414,269.84
393,393,Grass/straw,60.588183,23.446406,798.44


In [390]:
coord_df.to_excel("sites.xlsx")

In [391]:
#merged_df = df.applymap(lambda route: [coord_df.loc[idx, ['latitude', 'longitude']].tolist() for idx in route])
routes_with_dates = df

In [392]:
routes_with_dates['Day'] = routes_with_dates.reset_index().index+1

In [393]:
routes_with_dates

,Vehicle_1,Vehicle_2,Vehicle_3,Vehicle_4,Vehicle_5,Vehicle_6,Vehicle_7,Vehicle_8,Vehicle_9,Day
0,[],[],[],[],"[394, 129, 394]",[],[],[],"[394, 122, 394]",1
1,[],[],[],[],[],[],[],[],[],2
2,[],[],[],[],[],[],[],[],"[394, 103, 394]",3
3,[],[],[],[],[],[],[],[],[],4
4,[],[],[],[],[],[],[],[],[],5
...,...,...,...,...,...,...,...,...,...,...
246,[],[],[],[],"[394, 182, 394]",[],[],"[394, 45, 394]",[],247
247,[],[],[],[],"[394, 182, 394]",[],[],[],"[394, 311, 394]",248
248,[],[],[],[],[],[],"[394, 349, 394]",[],"[394, 178, 394]",249
249,[],[],[],[],[],[],[],[],[],250


In [394]:
routes_with_dates.to_excel("routes_with_dates.xlsx")

In [395]:
# Step 1: Create a boolean mask to identify rows with empty lists in all columns
empty_rows_mask = merged_df.apply(lambda row: all(isinstance(cell, list) and len(cell) == 0 for cell in row), axis=1)

# Step 2: Use the boolean mask to get the indexes of rows with empty lists
empty_row_indexes = (empty_rows_mask[empty_rows_mask].index+1).tolist()

# Step 3: Count the number of empty rows
num_empty_rows = len(empty_row_indexes)

# Step 4: Print the indexes and count
print("No routes for days:", empty_row_indexes)
print("Days of no driving required:", num_empty_rows)

No routes for days: [2, 4, 5, 6, 8, 13, 22, 32, 42, 74, 115, 120, 127, 132, 142, 147, 158, 176, 188, 202, 212, 220, 222, 228, 231, 235, 240, 244, 250]
Days of no driving required: 29


In [396]:
vehicle1 = routes_with_dates[['Vehicle_1', 'Day']]
vehicle2 = routes_with_dates[['Vehicle_2', 'Day']]
vehicle3 = routes_with_dates[['Vehicle_3', 'Day']]
vehicle4 = routes_with_dates[['Vehicle_4', 'Day']]
vehicle5 = routes_with_dates[['Vehicle_5', 'Day']]
vehicle6 = routes_with_dates[['Vehicle_6', 'Day']]
vehicle7 = routes_with_dates[['Vehicle_7', 'Day']]
vehicle8 = routes_with_dates[['Vehicle_8', 'Day']]
vehicle9 = routes_with_dates[['Vehicle_9', 'Day']]

In [397]:
mask1 = vehicle1['Vehicle_1'].apply(lambda x: len(x) == 0)
mask2 = vehicle2['Vehicle_2'].apply(lambda x: len(x) == 0)
mask3 = vehicle3['Vehicle_3'].apply(lambda x: len(x) == 0)
mask4 = vehicle4['Vehicle_4'].apply(lambda x: len(x) == 0)
mask5 = vehicle5['Vehicle_5'].apply(lambda x: len(x) == 0)
mask6 = vehicle6['Vehicle_6'].apply(lambda x: len(x) == 0)
mask7 = vehicle7['Vehicle_7'].apply(lambda x: len(x) == 0)
mask8 = vehicle8['Vehicle_8'].apply(lambda x: len(x) == 0)
mask9 = vehicle9['Vehicle_9'].apply(lambda x: len(x) == 0)

vehicle1 = vehicle1[~mask1]
vehicle2 = vehicle2[~mask2]
vehicle3 = vehicle3[~mask3]
vehicle4 = vehicle4[~mask4]
vehicle5 = vehicle5[~mask5]
vehicle6 = vehicle6[~mask6]
vehicle7 = vehicle7[~mask7]
vehicle8 = vehicle8[~mask8]
vehicle9 = vehicle9[~mask9]

In [398]:
vehicle1.to_excel("vehicle1.xlsx")
vehicle2.to_excel("vehicle2.xlsx")
vehicle3.to_excel("vehicle3.xlsx")
vehicle4.to_excel("vehicle4.xlsx")
vehicle5.to_excel("vehicle5.xlsx")
vehicle6.to_excel("vehicle6.xlsx")
vehicle7.to_excel("vehicle7.xlsx")
vehicle8.to_excel("vehicle8.xlsx")
vehicle9.to_excel("vehicle9.xlsx")